<a href="https://colab.research.google.com/github/isidoraconic/song-rec-system/blob/main/als_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alternating Least Squares (ALS) Demo 1

### Working with **implicit** data

In this example, we will be using matrix factorization in order to transform the interaction matrix (original matrix with users and items - and their preference of the items) into a more digestible, lower dimension matrix. The original matrix often has very many (millions) dimensions, but the actual "tastes" that drive recommendations are not as complex/ lower dimensionality.

Using matrix factorization, it is possible to mathematically reduce the dimensionality of the original "all users by all items" matrix into into something much smaller that represents “all items by some taste dimensions” and “all users by some taste dimensions”. These dimensions are called latent or hidden features and we learn them from our data. [Source.](https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe)

*If we can express each user as a vector of their taste values, and at the same time express each item as a vector of what tastes they represent. You can see we can quite easily make a recommendation. This also gives us the ability to find connections between users who have no specific items in common but share common tastes.*

Common types of matrix factorization include SVD (Singular Value Decomposition) and PLSA (Probailistic Latent Semantic Analysis), which work well for explicit data. With explicit data, unknown values can be treated simply as unknown, and we can impute/predict some values to them (i.e. a movie rating matrix). However, this is more difficult with implicit data - a missing value might mean that the user disliked a song, movie, etc. or simply that they have not seen it yet.

This is where Alternating Least Squares (ALS) is useful.

### Alternating Least Squares (ALS)
**Note that the below demo will be done on the last-fm dataset.**

*ALS is an iterative optimization process where we for every iteration try to arrive closer and closer to a factorized representation of our original data.*

In this first iteration, we will use the logic as outlined in [Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf), in which "their solution is to merge the preference (p) for an item with the confidence (c) we have for that preference. We start out with missing values as a negative preference with a low confidence value and existing values a positive preference but with a high confidence value. We can use something like play count, time spent on a page or some other form of interaction as the basis for calculating our confidence."

In [ ]:
# If using library, uncomment the lines below:
# !pip install implicit
# import implicit

In [ ]:
# Training the model would go something like:
# Initialize the als model and fit it using the sparse item-user matrix
# model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
# alpha_val = 15
# data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
# model.fit(data_conf)

In [ ]:
# Necessary imports
import random
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler

In [8]:
# Getting the last-fm-360K dataset
!wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz

--2022-02-01 19:44:52--  http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz
Resolving mtg.upf.edu (mtg.upf.edu)... 84.89.139.55
Connecting to mtg.upf.edu (mtg.upf.edu)|84.89.139.55|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 569202935 (543M) [application/x-gzip]
Saving to: ‘lastfm-dataset-360K.tar.gz.1’

lastfm-dataset-360K 100%[===================>] 542.83M   799KB/s    in 7m 53s  

2022-02-01 19:52:45 (1.15 MB/s) - ‘lastfm-dataset-360K.tar.gz.1’ saved [569202935/569202935]



In [9]:
!ls

lastfm-dataset-360K	    lastfm-dataset-360K.tar.gz.1
lastfm-dataset-360K.tar.gz  sample_data


In [10]:
# Unzip the folder:
!tar -xvzf lastfm-dataset-360K.tar.gz

lastfm-dataset-360K/
lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv
lastfm-dataset-360K/README.txt
lastfm-dataset-360K/mbox_sha1sum.py
lastfm-dataset-360K/usersha1-profile.tsv


In [11]:
# Check that it was unzipped
!ls

lastfm-dataset-360K	    lastfm-dataset-360K.tar.gz.1
lastfm-dataset-360K.tar.gz  sample_data


In [13]:
# Load the data into a dataframe
raw_data = pd.read_table('lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv')
raw_data = raw_data.drop(raw_data.columns[1], axis=1)
raw_data.columns = ['user', 'artist', 'plays']
raw_data

,user,artist,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
1,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
2,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
3,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
4,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
...,...,...,...
17535649,"sep 20, 2008",turbostaat,12
17535650,"sep 20, 2008",cuba missouri,11
17535651,"sep 20, 2008",little man tate,11
17535652,"sep 20, 2008",sigur rós,10


In [15]:
from google.colab import data_table
data_table.disable_dataframe_formatter()
# data_table.enable_dataframe_formatter()

In [16]:
raw_data

,user,artist,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
1,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
2,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
3,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
4,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
...,...,...,...
17535649,"sep 20, 2008",turbostaat,12
17535650,"sep 20, 2008",cuba missouri,11
17535651,"sep 20, 2008",little man tate,11
17535652,"sep 20, 2008",sigur rós,10


In [20]:
# A little bit of data filtering, drop NaN columns
# Also adding additional columns with categorical variables converted to numerical (user to user_id and artist to artist_id)
data = raw_data.dropna()
data = data.copy()
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")
data['user_id'] = data['user'].cat.codes
data['artist_id'] = data['artist'].cat.codes

In [21]:
data

,user,artist,plays,user_id,artist_id
0,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099,0,90933
1,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897,0,185367
2,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717,0,106704
3,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706,0,155241
4,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691,0,220128
...,...,...,...,...,...
17535649,"sep 20, 2008",turbostaat,12,358867,271740
17535650,"sep 20, 2008",cuba missouri,11,358867,78482
17535651,"sep 20, 2008",little man tate,11,358867,171784
17535652,"sep 20, 2008",sigur rós,10,358867,235118


In [22]:
# Importing libraries for creating the item-user matrix
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

In [23]:
# Create an item lookup table 
# We will only use user_id and artist_id in the matrix, so we want to be able to 
# look up the corresponding value in plain English later
# Some code from: https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe 
artist_lookup = data[['artist_id', 'artist']].drop_duplicates()
artist_lookup['artist_id'] = artist_lookup.artist_id.astype(str)

In [25]:
# Creating user lookup table (not super necessary but in case)
user_lookup = data[['user_id', 'user']].drop_duplicates()
user_lookup['user_id'] = user_lookup.user_id.astype(str)

In [26]:
# Now we drop the plain english user and artist columns
# We only want the numerical ID columns
data = data.drop(['user', 'artist'], axis=1)

In [27]:
 # Drop any rows that have 0 plays
 # But why would we do this - doesn't 0 plays tell us something? *****
 # data = data.loc[data.plays != 0]

In [29]:
# Create lists of all users, artists and plays
users = list(np.sort(data.user_id.unique()))
artists = list(np.sort(data.artist_id.unique()))
plays = list(data.plays)

In [34]:
# Get the rows and columns for our new matrix
rows = data.user_id.astype(int)
cols = data.artist_id.astype(int)

`csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])`
where `data`, `row_ind` and `col_ind` satisfy the relationship `a[row_ind[k], col_ind[k]] = data[k]`.
[SciPy Documentation about `scipy.sparse.csr_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)

In [35]:
# Contruct a sparse matrix for our users and items containing number of plays
# How does it know which value of plays corresponds to which cell? *****
data_sparse = sparse.csr_matrix((plays, (rows, cols)), shape=(len(users), len(artists)))

In [36]:
def als(sparse_matrix, alpha, iterations, lambda_val, features):
  """ Implementation of the Alternating Least Squares matrix factorization algorithm.
  
  Iteratively compute the U and V matrices by fixing one and optimizing the other (alternating), where their product approximates the original user-item matrix (sparse_matrix).
  This is done by alternatingly computing the user (x_u) and item (y_i) vectors using the following formulas:

  x_u = ((Y.T*Y + Y.T*(Cu - I) * Y) + lambda*I)^-1 * (X.T * Cu * p(u))
  y_i = ((X.T*X + X.T*(Ci - I) * X) + lambda*I)^-1 * (Y.T * Ci * p(i))

  Arguments:

    sparse_data (csr_matrix): Our sparse user-by-item matrix

    alpha (int): Rate at which the confidence increases. In the aforementioned paper, the authors found that 40 works well, and values between 15 and 40 are worth trying.

    iterations (int): The number of times to alternate between fixing and updating the user and item vectors. We will use 10 by default.

    lambda_val (float): Regularization value to reduce overfitting. We will use 0.1 by default.

    features (int): How many latent features we would like to compute. We will use 10 by default.

  Returns:
    
    X (csr_matrix): User vectors of size users-by-features.

    Y (csr_matrix): Item vectors of size items-by-features. In this case, this is artists.

  """

  # Default values being set. This can be commented out for other values.

  alpha=40
  iterations=10
  lambda_val=0.1
  features=10

  # Calculating confidence for each value in the input (sparse) matrix (sparse_matrix)
  # Adding +1 so that there is a minimal confidence even if alpha*cell in matrix = 0 
  # (i.e. if there are 0 plays of a particular song) 
  # If I had dropped the rows with 0 plays, I would not need to do this?
  confidence = 1 + alpha*sparse_matrix 

  # Get the size of the user rows and item columns (artists)
  num_users, num_items = sparse_matrix.shape

  # Calulating the preference for each value in the input (sparse) matrix (sparse_matrix)
  # Not sure if this is necessary (also exactly how to format) because we can directly impute
  preference = sparse.csr_matrix((num_users, num_items))

  # Create X (user) and Y (item - in this case artist) matrices
  # Randomly assign them values - we will alternatingly update each of the values as per equations above
  # Note: np.random.normal(size = (num_users, features)) --> Creates a num_users x features matrix with
  # randomly assigned values which are normalized (between -1 and 1)
  # This numpy 2D matrix is then passed into the csr_matrix constructor
  X = sparse.csr_matrix(np.random.normal(size = (num_users, features)))
  Y = sparse.csr_matrix(np.random.normal(size = (num_items, features)))

  # Pre-computing the identity matrices
  # Using the scipy.sparse.eye function which returns a sparse (mxn) matrix where the kth diagonal
  # is all ones and everything else is zeroes
  # https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.eye.html 
  # ***** Why not make it an num_users x features matrix?
  X_I = sparse.eye(num_users)
  Y_I = sparse.eye(num_items)

  # Feature identity matrix
  I = sparse.eye(features)

  # Regularized (lambda) feature identity matrix
  # **** Where is this? 
  lI = lambda_val*I

  # COMPUTATION BEGINS HERE
  # We are doing the number of iterations specified in the args (default is 10)
  # In each iteration: 
  # (1) We first precompute X_T_X and Y_T_Y (X-transpose-X and Y-transpose-Y)
  # (2) Calculate all x values (all user values)
  # (3) Calculate all y values (all item - artist values)
  for i in range(iterations):
    print("Iteration #", i+1, "of ", iterations, "iterations.")

    # Precomputing X-transpose-X and Y-transpose-Y
    X_T_X = (X.T).dot(X)
    Y_T_Y = (Y.T).dot(Y)

    # Computing all x values in X (i.e all users)
    # Treat them one row at a time because row = user
    for u in range(num_users):

      # Get this user row
      # this means get row u, and : means all columns of u
      # ***** Why are we not getting the user row from the randomly assigned matrix X?
      user_row = confidence[u,:].toarray()

      # Getting the preference of this row/user
      # It is binary; if the value is greater than 0, then the preference = 1
      # Otherwise the preference = 0
      # ***** BUT, since we are using the confidence values, won't this never be 0 because we added the +1?
      p_user = user_row.copy()
      p_user[p_user != 0] = 1.0

      # Calculating Cu and Cu - I
      # sparse.diags --> constructs a sparse matrix from the diagonal
      # In this case, the user row vector values will be along the main diagonal, no offset (specified as 0)
      CuI = sparse.diags(user_row, [0]) # --> Where in the formula is this?
      Cu = CuI + Y_I


  (0, 19356)	229
  (0, 19592)	288
  (0, 37515)	310
  (0, 45554)	135
  (0, 46244)	134
  (0, 51085)	185
  (0, 90933)	1099
  (0, 92172)	154
  (0, 100639)	302
  (0, 103711)	232
  (0, 106689)	222
  (0, 106704)	717
  (0, 125484)	134
  (0, 126690)	361
  (0, 129383)	151
  (0, 137062)	358
  (0, 144238)	227
  (0, 144293)	316
  (0, 151766)	182
  (0, 154684)	198
  (0, 155241)	706
  (0, 165120)	135
  (0, 167984)	281
  (0, 169906)	387
  (0, 171686)	145
  :	:
  (358867, 160643)	14
  (358867, 161415)	27
  (358867, 161724)	14
  (358867, 162097)	32
  (358867, 167143)	131
  (358867, 171784)	11
  (358867, 178275)	12
  (358867, 186549)	18
  (358867, 217883)	12
  (358867, 218091)	23
  (358867, 235118)	10
  (358867, 251158)	16
  (358867, 254306)	36
  (358867, 254532)	12
  (358867, 258344)	14
  (358867, 262827)	10
  (358867, 263239)	12
  (358867, 263330)	30
  (358867, 263682)	14
  (358867, 263831)	24
  (358867, 265438)	12
  (358867, 267931)	25
  (358867, 268672)	45
  (358867, 271740)	12
  (358867, 274800)	39
